##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 Quickstart für Anfänger

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Diese Einführung verwendet [Keras](https://www.tensorflow.org/guide/keras/overview) für die nachfolgenden Schritte:

1. Laden eines vorgefertigten Datensatzes.
2. Erstellen eines maschinellen Lernmodells mit einem neuronalen Netzwerk, das Bilder klassifiziert.
3. Trainieren dieses neuronalen Netzwerks.
4. Evaluieren der Genauigkeit des Modells.

## TensorFlow einrichten

Importiere TensorFlow, um loszulegen. TensorFlow ist eine weit verbreitete Open-Source-Bibliothek, die von Google entwickelt wurde und für maschinelles Lernen (ML) und künstliche Intelligenz (KI) verwendet wird. TensorFlow bietet viele Funktionalitäten für die Entwicklung und den Einsatz von ML-Modellen.

In [ ]:
# TensorFlow und tf.keras
import tensorflow as tf

# Hilfsbibliotheken
import matplotlib.pyplot as plt

print("TensorFlow Version:", tf.__version__)

## Laden eines Datensatzes

Lade den [MNIST-Datensatz] (http://yann.lecun.com/exdb/mnist/) und bereite ihn vor. MNIST ist ein umfangreicher Datensatz, der Bilder von handgeschriebenen Ziffern (0 bis 9) enthält. Der Datensatz ist in zwei Teile unterteilt. Einen zum Trainieren und einen zum Testen des Modells. Der Datensatz besteht aus Bilder und Labels. Ein Label beschreibt die Klasse, zu der eine bestimmtes Bild zugeordnet wird (Bei MNIST einer Zahl). Es dient als Zielklasse, die das Modell während des Trainings lernen soll.

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() # x sind die Bilder, y sind die zugehörigen Label

Wir können den Inhalt eines solchen Bildes als Beispiel ausgeben lassen. Die entsprechende Beschriftung sollte mit dem Bild übereinstimmen.

In [ ]:
# Auswahl des dritten Bildes (der Index beginnt in Python bei 0) aus allen 60.000 Trainingsbildern
example_image = x_train[2]

# Darstellen der 28x28 pixel großen Ziffern in Graustufen mit der integrierten Funktion imshow()
plt.imshow(example_image, cmap=plt.cm.binary)

# Wir können sehen, dass das Trainingslabel des Bildes = 4 korrekt ist
print('label:', y_train[2])

Die Pixelwerte der Bilder reichen von 0 bis 255. Wir skalieren diese Werte auf einen Bereich von 0 bis 1, indem wir die Werte durch „255.0“ teilen. Dies wird auch als Normierung bezeichnet und wandelt die Beispieldaten von Ganzzahlen in Gleitkommazahlen um:

In [ ]:
print(x_train.dtype) # Variablentyp
x_train, x_test = x_train / 255.0, x_test / 255.
print(x_train.dtype)

## Erstellen eines Modells für maschinelles Lernen

Nun wollen wir ein `tf.keras.Sequential`-Modell erstellen. Ein Modell besteht aus verschiedenen Layern (Schichten). Layer sind Funktionen mit einer bekannten mathematischen Struktur, die wiederverwendet werden können und trainierbare Variablen haben. [`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) ist nützlich, um Layer zu stapeln, wobei jedes Layer einen Eingabe-[Tensor](https://www.tensorflow.org/guide/tensor) und einen Ausgabe-Tensor hat. Dieses Modell verwendet die Layer [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), und [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout).

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Für jedes Beispiel liefert das Modell einen Vektor von [logits](https://developers.google.com/machine-learning/glossary#logits) oder [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) Werten. Für MNIST erhalten wir 10 Werte, einen für jede Klasse (0-9). Jeder Wert gibt an, wie stark das Modell davon ausgeht, dass das Eingabebeispiel zu dieser Klasse gehört.

In [ ]:
predictions = model(x_train[:1]).numpy()
print(predictions)

Da diese Werte nicht skaliert sind, benötigen wir eine weitere Funktion, um sie zu skalieren. Die Funktion „tf.nn.softmax“ wird verwendet, um diese Werte in *Wahrscheinlichkeiten* (Werte zwischen 0 und 1) für jede Klasse umzuwandeln. Eine Wahrscheinlichkeit von 1 würde bedeuten, dass das Modell zu 100 % sicher ist, dass das Beispiel zu dieser Klasse gehört. 

In [ ]:
probabilitites = tf.nn.softmax(predictions).numpy()
print(probabilitites)

Im nächsten Schritt wollen wir eine loss-Funktion (Verlustfunktion) für das Training definieren. Eine loss-Funktion misst die Differenz zwischen den vorhergesagten Werten des Modells und den tatsächlichen Zielwerten. Der Hauptzweck einer loss-Funktion besteht darin, das Training des Modells zu steuern, indem ein Wert berechnet wird, der angibt, wie gut das Modell seine Vorhersagen bereits trifft. Das Modell wird dann so trainiert, dass der Verlust minimiert wird. In diesem Beispiel verwenden wir `losses.SparseCategoricalCrossentropy` als loss-Funktion:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Bevor wir mit dem Training beginnen, konfigurieren und kompilieren wir das Modell mit Keras `Model.compile`. Hierfür setzen wir die Klasse [`Optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) auf `Adam`. Ein Optimizer (Optimierer) ist ein Algorithmus, der die internen Parameter des Modells während des Trainings so anpasst, dass der Verlust minimal wird. Wir setzen `loss` auf die Funktion `loss_fn`, die wir zuvor definiert haben, und geben eine Metrik an, die für das Modell bewertet werden soll, indem  wir den Parameter `metrics` auf `accuracy` setzen.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Trainieren und evaluieren des Modells

Verwenden die Methode `Model.fit`, um das Training zu starten. Dadurch werden die Modellparameter angepasst und der Verlust minimiert. Das Training findet in mehreren Epochen statt. Innerhalb einer Epoche wird das Modell einmal auf allen Trainingsdaten trainiert. Dazu werden die Trainingsdaten in Minibatches aufgeteilt. Die Batchgröße (Größe der Minibatches) wird verwendet, um zu bestimmen, wie viele Trainingsdaten das Modell auf einmal verarbeiten soll.

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Die Balken zeigen an, wie viele Iterationen innerhalb einer Epoche durchgeführt wurden. Eine Iteration ist der Vorgang, bei dem das Modell einen Minibatch durchläuft. Wir können also die Iterationen einer Epoche berechnen aus:
Iterationen = Trainingsdaten / Batchsize = 60000 / 32 = 1875

Nachdem das Modell trainiert wurde, können wir es nun auswerten. Die Methode `Model.evaluate` prüft die Leistung des Modells, in der Regel anhand eines [validation set](https://developers.google.com/machine-learning/glossary#validation-set) oder [test set](https://developers.google.com/machine-learning/glossary#test-set)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

Das Modell zur Bildklassifizierung ist nun auf ~98% Genauigkeit auf diesem Datensatz trainiert. Um mehr zu erfahren, lese die [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

## Fazit

Herzlichen Glückwunsch! Du hast ein Modell für maschinelles Lernen mit einem vorgefertigten Datensatz unter Verwendung der [Keras](https://www.tensorflow.org/guide/keras/overview) API trainiert.

Weitere Beispiele zur Verwendung von Keras findest du in den [tutorials](https://www.tensorflow.org/tutorials/keras/). Um mehr über die Erstellung von Modellen mit Keras zu erfahren, lese die [guides](https://www.tensorflow.org/guide/keras). Wenn du mehr über das Laden und Aufbereiten von Daten erfahren möchten, lese die Tutorials zu [Laden von Bilddaten](https://www.tensorflow.org/tutorials/load_data/images) oder [Laden von CSV-Daten](https://www.tensorflow.org/tutorials/load_data/csv).